In [5]:
import pandas as pd
import numpy as np
import os


In [6]:
pre_path = "./raw_data/"
path = "example_data.csv"
#abs_path = os.path.abspath(pre_path + path)

df = pd.read_csv("../raw_data/"+path)
df.head()

,Unnamed: 0,0,topic
0,0,2 2 0 2 g u A 6 2 ] E H . h p - o r t s a [ ...,Astrophysics
1,1,"Draft version August 29, 2022 Typeset using LA...",Astrophysics
2,2,Astronomy & Astrophysics manuscript no. 41891c...,Astrophysics
3,3,Astronomy & Astrophysics manuscript no. aa Aug...,Astrophysics
4,4,2 2 0 2 g u A 6 2 ] R S . h p - o r t s a [...,Astrophysics
